In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error

In [2]:
data = pd.read_csv('car data.csv')

In [3]:
data.head()

Car_Name  Year  Selling_Price  Present_Price  Kms_Driven Fuel_Type  \
0     ritz  2014           3.35           5.59       27000    Petrol   
1      sx4  2013           4.75           9.54       43000    Diesel   
2     ciaz  2017           7.25           9.85        6900    Petrol   
3  wagon r  2011           2.85           4.15        5200    Petrol   
4    swift  2014           4.60           6.87       42450    Diesel   

  Seller_Type Transmission  Owner  
0      Dealer       Manual      0  
1      Dealer       Manual      0  
2      Dealer       Manual      0  
3      Dealer       Manual      0  
4      Dealer       Manual      0

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Car_Name       301 non-null    object 
 1   Year           301 non-null    int64  
 2   Selling_Price  301 non-null    float64
 3   Present_Price  301 non-null    float64
 4   Kms_Driven     301 non-null    int64  
 5   Fuel_Type      301 non-null    object 
 6   Seller_Type    301 non-null    object 
 7   Transmission   301 non-null    object 
 8   Owner          301 non-null    int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 21.3+ KB


In [5]:
data.shape

(301, 9)

In [6]:
data.isnull().sum()

Car_Name         0
Year             0
Selling_Price    0
Present_Price    0
Kms_Driven       0
Fuel_Type        0
Seller_Type      0
Transmission     0
Owner            0
dtype: int64

In [7]:
data.nunique()

Car_Name          98
Year              16
Selling_Price    156
Present_Price    147
Kms_Driven       206
Fuel_Type          3
Seller_Type        2
Transmission       2
Owner              3
dtype: int64

In [8]:
data["Owner"].replace([0,1,3],["One_owner","Two_owner","Four_owner"],inplace=True)

In [9]:
data.duplicated().sum()

2

In [10]:
np.where(data.duplicated()==True)[0]

array([17, 93], dtype=int64)

In [11]:
data.iloc[[17,93],:]

Car_Name  Year  Selling_Price  Present_Price  Kms_Driven Fuel_Type  \
17    ertiga  2016           7.75          10.79       43000    Diesel   
93  fortuner  2015          23.00          30.61       40000    Diesel   

   Seller_Type Transmission      Owner  
17      Dealer       Manual  One_owner  
93      Dealer    Automatic  One_owner

In [12]:
data.drop_duplicates(inplace=True)

In [13]:
data.shape

(299, 9)

In [14]:
df = pd.DataFrame(data.values,columns=data.columns)
df.shape

(299, 9)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Car_Name       299 non-null    object
 1   Year           299 non-null    object
 2   Selling_Price  299 non-null    object
 3   Present_Price  299 non-null    object
 4   Kms_Driven     299 non-null    object
 5   Fuel_Type      299 non-null    object
 6   Seller_Type    299 non-null    object
 7   Transmission   299 non-null    object
 8   Owner          299 non-null    object
dtypes: object(9)
memory usage: 21.1+ KB


In [16]:
df["Year"] = df["Year"].astype(np.int64)
df["Selling_Price"] = df["Selling_Price"].astype(np.float64)
df["Present_Price"] = df["Present_Price"].astype(np.float64)
df["Kms_Driven"] = df["Kms_Driven"].astype(np.int64)
df["Fuel_Type"] = df["Fuel_Type"].astype("category")
df["Seller_Type"] = df["Seller_Type"].astype("category")
df["Transmission"] = df["Transmission"].astype("category")
df["Owner"] = df["Owner"].astype("category")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   Car_Name       299 non-null    object  
 1   Year           299 non-null    int64   
 2   Selling_Price  299 non-null    float64 
 3   Present_Price  299 non-null    float64 
 4   Kms_Driven     299 non-null    int64   
 5   Fuel_Type      299 non-null    category
 6   Seller_Type    299 non-null    category
 7   Transmission   299 non-null    category
 8   Owner          299 non-null    category
dtypes: category(4), float64(2), int64(2), object(1)
memory usage: 13.5+ KB


In [17]:
del df["Car_Name"]

In [18]:
df.describe().T

count          mean           std      min       25%       50%  \
Year           299.0   2013.615385      2.896868  2003.00   2012.00   2014.00   
Selling_Price  299.0      4.589632      4.984240     0.10      0.85      3.51   
Present_Price  299.0      7.541037      8.567887     0.32      1.20      6.10   
Kms_Driven     299.0  36916.752508  39015.170352   500.00  15000.00  32000.00   

                    75%       max  
Year            2016.00    2018.0  
Selling_Price      6.00      35.0  
Present_Price      9.84      92.6  
Kms_Driven     48883.50  500000.0

In [19]:
df.head(5)

Year  Selling_Price  Present_Price  Kms_Driven Fuel_Type Seller_Type  \
0  2014           3.35           5.59       27000    Petrol      Dealer   
1  2013           4.75           9.54       43000    Diesel      Dealer   
2  2017           7.25           9.85        6900    Petrol      Dealer   
3  2011           2.85           4.15        5200    Petrol      Dealer   
4  2014           4.60           6.87       42450    Diesel      Dealer   

  Transmission      Owner  
0       Manual  One_owner  
1       Manual  One_owner  
2       Manual  One_owner  
3       Manual  One_owner  
4       Manual  One_owner

In [21]:
print(df[["Owner","Seller_Type"]].value_counts())

Owner       Seller_Type
One_owner   Dealer         189
            Individual      99
Two_owner   Individual       6
            Dealer           4
Four_owner  Individual       1
dtype: int64


In [22]:
df.head(4)


Year  Selling_Price  Present_Price  Kms_Driven Fuel_Type Seller_Type  \
0  2014           3.35           5.59       27000    Petrol      Dealer   
1  2013           4.75           9.54       43000    Diesel      Dealer   
2  2017           7.25           9.85        6900    Petrol      Dealer   
3  2011           2.85           4.15        5200    Petrol      Dealer   

  Transmission      Owner  
0       Manual  One_owner  
1       Manual  One_owner  
2       Manual  One_owner  
3       Manual  One_owner

In [23]:
df["Number_of_years_old"] = 2023 - df["Year"]
df.head(5)

Year  Selling_Price  Present_Price  Kms_Driven Fuel_Type Seller_Type  \
0  2014           3.35           5.59       27000    Petrol      Dealer   
1  2013           4.75           9.54       43000    Diesel      Dealer   
2  2017           7.25           9.85        6900    Petrol      Dealer   
3  2011           2.85           4.15        5200    Petrol      Dealer   
4  2014           4.60           6.87       42450    Diesel      Dealer   

  Transmission      Owner  Number_of_years_old  
0       Manual  One_owner                    9  
1       Manual  One_owner                   10  
2       Manual  One_owner                    6  
3       Manual  One_owner                   12  
4       Manual  One_owner                    9

In [24]:
del df["Year"]


In [25]:
col_to_transform = ["Selling_Price","Present_Price","Kms_Driven","Number_of_years_old"]
print(df[col_to_transform].skew())


Selling_Price          2.536522
Present_Price          4.186894
Kms_Driven             6.418134
Number_of_years_old    1.236876
dtype: float64


In [26]:
col_to_label_encode = ["Seller_Type","Transmission"]
for col in col_to_label_encode:
 label=LabelEncoder()
 label.fit(df[col])
 df[col]=label.transform(df[col])


In [27]:
col_to_oneencode = ["Fuel_Type"]
for col in col_to_oneencode:
 ohe=OneHotEncoder(sparse=False)
 matrix=ohe.fit(df[[col]])
 matrix=ohe.transform(df[[col]])
 matrix=pd.DataFrame(matrix,columns=[col[:5]+"_"+str(val) for val in ohe.categories_[0]])
 df=pd.concat([df,matrix],axis=1)


In [28]:
col_to_del = ["Fuel_Type"]
for col in col_to_del:
 del df[col]


In [29]:
df["Owner"].replace(["One_owner","Two_owner","Four_owner"],[1,2,4],inplace=True)


In [30]:
X = df.iloc[:,1:]
Y = df.iloc[:,0]
print("Shape of X : ",X.shape)
print("Shape of Y : ",Y.shape)


Shape of X :  (299, 9)
Shape of Y :  (299,)


In [31]:
corr_matrix = X.corr()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape),k=1).astype(np.bool_))
to_drop = [column for column in upper.columns if any(abs(upper[column])>0.85)]
print("Columns of to drop : ",to_drop)
X = X.drop(X[to_drop],axis=1)
print("\nIndependent Features are : ",X.columns)


Columns of to drop :  ['Fuel__Petrol']

Independent Features are :  Index(['Present_Price', 'Kms_Driven', 'Seller_Type', 'Transmission', 'Owner',
       'Number_of_years_old', 'Fuel__CNG', 'Fuel__Diesel'],
      dtype='object')


In [32]:
scaler = StandardScaler()
Xscaled = scaler.fit_transform(X)


In [33]:
Xtrain,Xtest,Ytrain,Ytest = train_test_split(Xscaled,Y,test_size=0.2,shuffle=True,random_state=10)
print("X train shape : ",Xtrain.shape)
print("X test shape : ",Xtest.shape)
print("Y train shape : ",Ytrain.shape)
print("Y test shape : ",Ytest.shape)


X train shape :  (239, 8)
X test shape :  (60, 8)
Y train shape :  (239,)
Y test shape :  (60,)


In [34]:
MSE_ = []
RMSE_ = []
MAE_ = []
CV = []
R2_score_test_ = []


In [36]:
error = []
for i in range(2,10):
 knn = KNeighborsRegressor(n_neighbors=i)
 knn.fit(Xtrain,Ytrain)
 k = 1-cross_val_score(knn, Xtrain, Ytrain, cv=5).mean()
 error.append(k)
 print(f"The value of k is {i} and the error is : {k}")
 



The value of k is 2 and the error is : 0.120995163884411
The value of k is 3 and the error is : 0.11985640169661205
The value of k is 4 and the error is : 0.11770616728911798
The value of k is 5 and the error is : 0.1220535109851284
The value of k is 6 and the error is : 0.13933746908918754
The value of k is 7 and the error is : 0.14884463545982152
The value of k is 8 and the error is : 0.1506152045667104
The value of k is 9 and the error is : 0.15815723572230167


In [37]:
#  Building the model
knn = KNeighborsRegressor(n_neighbors=3)
# Training the model
knn.fit(Xtrain,Ytrain)

KNeighborsRegressor(n_neighbors=3)

In [38]:
cross_val = cross_val_score(knn,Xtrain,Ytrain,cv=5) # calculating training accuracy with K-Fold 
cross_val = cross_val.mean()
Ypred = knn.predict(Xtest)
MSE = mean_squared_error(Ytest,Ypred) # calculating MSE
RMSE = np.sqrt(mean_squared_error(Ytest,Ypred)) # calculating the RMSE
MAE = mean_absolute_error(Ytest,Ypred) # calculating the MAE
R2_score_test = r2_score(Ytest,Ypred) # calculating the R2 score
print("Mean squared error in testing is : ",round(MSE,4))
print("Root Mean squared error in testing is : ",round(RMSE,4))
print("Mean absolute error in testing is : ",round(MAE,4))
print("CV score for training is : ",round(cross_val,3))
print("R2 score in testing is : ",round(R2_score_test,3))
MSE_.append(round(MSE,4))
RMSE_.append(round(RMSE,4))
MAE_.append(round(MAE,4))
CV.append(round(cross_val,3))
R2_score_test_.append(round(R2_score_test,3))


Mean squared error in testing is :  2.6561
Root Mean squared error in testing is :  1.6298
Mean absolute error in testing is :  0.7889
CV score for training is :  0.88
R2 score in testing is :  0.832


In [39]:
forest_model = RandomForestRegressor(random_state=10)
forest_model.fit(Xtrain,Ytrain)


RandomForestRegressor(random_state=10)

In [40]:
cross_val = cross_val_score(forest_model,Xtrain,Ytrain,cv=5)
cross_val = cross_val.mean()
Ypred = forest_model.predict(Xtest)
MSE = mean_squared_error(Ytest,Ypred) # calculating MSE
RMSE = np.sqrt(mean_squared_error(Ytest,Ypred)) # calculating the RMSE
MAE = mean_absolute_error(Ytest,Ypred) # calculating the MAE
R2_score_test = r2_score(Ytest,Ypred) # calculating the R2 score
print("Mean squared error in testing is : ",round(MSE,4))
print("Root Mean squared error in testing is : ",round(RMSE,4))
print("Mean absolute error in testing is : ",round(MAE,4))
print("CV score for training is : ",round(cross_val,3))
print("R2 score in testing is : ",round(R2_score_test,3))
MSE_.append(round(MSE,4))
RMSE_.append(round(RMSE,4))
MAE_.append(round(MAE,4))
CV.append(round(cross_val,3))
R2_score_test_.append(round(R2_score_test,3))


Mean squared error in testing is :  0.6354
Root Mean squared error in testing is :  0.7971
Mean absolute error in testing is :  0.4821
CV score for training is :  0.889
R2 score in testing is :  0.96


In [41]:
gradient_boosting = GradientBoostingRegressor(random_state=10)
gradient_boosting.fit(Xtrain,Ytrain)

GradientBoostingRegressor(random_state=10)

In [42]:
cross_val = cross_val_score(gradient_boosting,Xtrain,Ytrain,cv=5)
cross_val = cross_val.mean()
Ypred = gradient_boosting.predict(Xtest)
MSE = mean_squared_error(Ytest,Ypred) # calculating MSE
RMSE = np.sqrt(mean_squared_error(Ytest,Ypred)) # calculating the RMSE
MAE = mean_absolute_error(Ytest,Ypred) # calculating the MAE
R2_score_test = r2_score(Ytest,Ypred) # calculating the R2 score
print("Mean squared error in testing is : ",round(MSE,4))
print("Root Mean squared error in testing is : ",round(RMSE,4))
print("Mean absolute error in testing is : ",round(MAE,4))
print("CV score for training is : ",round(cross_val,3))
print("R2 score in testing is : ",round(R2_score_test,3))
MSE_.append(round(MSE,4))
RMSE_.append(round(RMSE,4))
MAE_.append(round(MAE,4))
CV.append(round(cross_val,3))
R2_score_test_.append(round(R2_score_test,3))

Mean squared error in testing is :  0.7082
Root Mean squared error in testing is :  0.8415
Mean absolute error in testing is :  0.4752
CV score for training is :  0.904
R2 score in testing is :  0.955


In [43]:
Model = ["KNeighborsRegressor","RandomForestRegressor","GradientBoostingRegressor"]
results=pd.DataFrame({
 'Model': Model,
'CV score for training': CV,
'R2 score test': R2_score_test_,
'MSE':MSE_,
'RMSE':RMSE_,
'MAE':MAE_,})
results.head()


Model  CV score for training  R2 score test     MSE  \
0        KNeighborsRegressor                  0.880          0.832  2.6561   
1      RandomForestRegressor                  0.889          0.960  0.6354   
2  GradientBoostingRegressor                  0.904          0.955  0.7082   

     RMSE     MAE  
0  1.6298  0.7889  
1  0.7971  0.4821  
2  0.8415  0.4752